# Assignment 1: Caption-Question Relevance Classification
## Advanced Baselines for NLP

**Task**: Binary classification to determine if a question is related to its corresponding caption

**Assignment Requirements**:
- ✅ Dataset: Min. 5,000 train / 1,000 test examples
- ✅ Stratified train/test split with random_state=42
- ✅ 5-Fold Stratified Cross-Validation
- ✅ Primary Metric: F1-Macro
- ✅ Feature Representations: TF-IDF (sparse) + Word Embeddings (dense)
- ✅ Ablation Studies: N-grams, Preprocessing, Hyperparameters
- ✅ Error Analysis: Confusion Matrix, Feature Analysis, Failure Cases

**Methodology Improvements**:
- 🎯 **Semantic Distance-Based Negative Sampling**: Uses FastText embeddings of image tags to create semantically distant negative pairs, preventing false negatives
- 🎯 **Enhanced Context**: Concatenates Question + Answer for richer semantic features
- 🎯 **Tag-Based Embeddings**: Leverages image tags for robust semantic distance computation

## ⚠️ Prerequisites

**Before running this notebook**, ensure you have:
1. ✅ Run `convert_jsonl_to_parquet.ipynb` to generate parquet files **with tags included**
2. ✅ The following files exist in your workspace:
   - `RSVLM-QA-captions.parquet` (with columns: id, image, caption, **tags**)
   - `RSVLM-QA-questions.parquet` (with columns: id, question_type, question, answer)

This notebook uses the **tags** field to compute semantic distances for improved negative sampling.

## 1. Import Required Libraries

In [1]:
# Data manipulation
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# NLP and text processing
import re
import string
from collections import Counter

# Scikit-learn for ML
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    classification_report, confusion_matrix, f1_score, 
    accuracy_score, precision_score, recall_score, 
    ConfusionMatrixDisplay
)

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Word embeddings
import gensim
from gensim.models import Word2Vec, KeyedVectors
import gensim.downloader as api

# Set random seed for reproducibility (Assignment Requirement)
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("✓ All libraries imported successfully!")

✓ All libraries imported successfully!


## 2. Load Parquet Data

In [2]:
# Load the two parquet files
df_captions = pd.read_parquet("RSVLM-QA-captions.parquet")
df_qa = pd.read_parquet("RSVLM-QA-questions.parquet")

print(f"Captions data shape: {df_captions.shape}")
print(f"QA pairs data shape: {df_qa.shape}")
print(f"\nCaptions columns: {df_captions.columns.tolist()}")
print(f"QA columns: {df_qa.columns.tolist()}")
print(f"\nNumber of unique images: {df_captions['id'].nunique()}")
print(f"Average QA pairs per image: {len(df_qa) / len(df_captions):.2f}")
print(f"\nTags available: {'tags' in df_captions.columns}")
if 'tags' in df_captions.columns:
    print(f"Average tags per image: {df_captions['tags'].apply(len).mean():.2f}")

Captions data shape: (13820, 4)
QA pairs data shape: (148558, 4)

Captions columns: ['id', 'image', 'caption', 'tags']
QA columns: ['id', 'question_type', 'question', 'answer']

Number of unique images: 13820
Average QA pairs per image: 10.75

Tags available: True
Average tags per image: 10.62


## 3. Create Binary Classification Dataset

We'll create a balanced dataset with:
- **Positive examples (label=1)**: Correct caption-question-answer triplets from the same image
- **Negative examples (label=0)**: Semantically distant caption-question-answer pairs

**Improvement**: Instead of random shuffling, we use tag-based semantic distance to pair captions with distant questions, preventing false negatives from similar content being paired together.

### 3.1 Load FastText Embeddings for Tag Vectorization

In [ ]:
print("Loading FastText embeddings for tag vectorization...")
print("This may take a minute on first download...")

try:
    # Try to load FastText embeddings
    fasttext_model = api.load('fasttext-wiki-news-subwords-300')
    print(f"✓ Loaded FastText embeddings: {len(fasttext_model)} words, {fasttext_model.vector_size} dimensions")
except Exception as e:
    print(f"! Unable to download FastText: {e}")
    print("! Using fallback: Loading smaller GloVe embeddings...")
    # Fallback to GloVe if FastText fails
    fasttext_model = api.load('glove-wiki-gigaword-100')
    print(f"✓ Loaded GloVe embeddings: {len(fasttext_model)} words, {fasttext_model.vector_size} dimensions")

vector_size = fasttext_model.vector_size

Loading FastText embeddings for tag vectorization...
This may take a minute on first download...


### 3.2 Compute Tag Embeddings for Each Image

In [ ]:
def get_tag_embedding(tags, model, vector_size):
    """
    Convert a list of tags to an averaged embedding vector.
    Each tag may consist of multiple words.
    """
    if not tags or len(tags) == 0:
        return np.zeros(vector_size)
    
    tag_vectors = []
    for tag in tags:
        # Split multi-word tags and get embeddings for each word
        words = str(tag).lower().replace('-', ' ').replace('_', ' ').split()
        word_vecs = []
        for word in words:
            if word in model:
                word_vecs.append(model[word])
        
        # Average words within the tag
        if len(word_vecs) > 0:
            tag_vectors.append(np.mean(word_vecs, axis=0))
    
    # Average all tag vectors
    if len(tag_vectors) == 0:
        return np.zeros(vector_size)
    
    return np.mean(tag_vectors, axis=0)

print("Computing tag embeddings for all images...")
df_captions['tag_embedding'] = df_captions['tags'].apply(
    lambda tags: get_tag_embedding(tags, fasttext_model, vector_size)
)

# Convert to numpy array for distance calculations
tag_embeddings = np.vstack(df_captions['tag_embedding'].values)
print(f"✓ Tag embeddings shape: {tag_embeddings.shape}")
print(f"✓ Sample tag embedding (first 5 dimensions): {tag_embeddings[0][:5]}")

### 3.3 Create Positive Examples (Correct Pairs)

In [ ]:
# Merge captions with QA pairs to create positive examples
# Now concatenate question + answer for more context
df_positive = df_qa.merge(df_captions[['id', 'caption']], on='id', how='inner')
df_positive['question_answer'] = df_positive['question'] + ' ' + df_positive['answer']
df_positive['label'] = 1  # Related caption-question pairs

print(f"Positive examples created: {len(df_positive)}")
print(f"\nSample positive example:")
sample = df_positive.iloc[0]
print(f"Caption: {sample['caption'][:150]}...")
print(f"Question: {sample['question']}")
print(f"Answer: {sample['answer'][:100]}...")
print(f"Question+Answer: {sample['question_answer'][:150]}...")
print(f"Label: {sample['label']} (Related)")

Positive examples created: 148558

Sample positive example:
Caption: The image depicts a highly developed urban area characterized by a prominent highway interchange that dominates the central portion of the scene. Surr...
Question: Where is the highway interchange located in the image?
Label: 1 (Related)


### 3.4 Create Negative Examples Using Semantic Distance

Instead of random shuffling, we match each QA pair with captions from semantically distant images based on tag embeddings. This prevents false negatives where similar content is incorrectly labeled as unrelated.

In [ ]:
# Create negative examples using semantic distance based on tag embeddings
from sklearn.metrics.pairwise import cosine_distances

print("Computing pairwise distances between tag embeddings...")
# Compute cosine distances between all tag embeddings
distance_matrix = cosine_distances(tag_embeddings)
print(f"✓ Distance matrix shape: {distance_matrix.shape}")

# Create a mapping from id to index in the distance matrix
id_to_idx = {id_val: idx for idx, id_val in enumerate(df_captions['id'].values)}

print("\nCreating negative examples by pairing with distant captions...")
df_negative = df_qa.copy()
distant_captions = []

for _, row in df_qa.iterrows():
    original_id = row['id']
    
    if original_id in id_to_idx:
        # Get the index of this image's embedding
        orig_idx = id_to_idx[original_id]
        
        # Get distances to all other images
        distances = distance_matrix[orig_idx]
        
        # Select from the top 25% most distant images to ensure diversity
        # Sort by distance (descending) and pick randomly from top quartile
        distant_indices = np.argsort(distances)[-len(distances)//4:]
        
        # Randomly select one from the distant images
        selected_idx = np.random.choice(distant_indices)
        distant_caption = df_captions.iloc[selected_idx]['caption']
        distant_captions.append(distant_caption)
    else:
        # Fallback: random caption if ID not found
        distant_captions.append(df_captions['caption'].sample(1, random_state=RANDOM_STATE).iloc[0])

df_negative['caption'] = distant_captions
df_negative['question_answer'] = df_negative['question'] + ' ' + df_negative['answer']
df_negative['label'] = 0  # Unrelated caption-question pairs

print(f"✓ Negative examples created: {len(df_negative)}")
print(f"\nSample negative example (distant pairing):")
sample = df_negative.iloc[0]
print(f"Caption: {sample['caption'][:150]}...")
print(f"Question: {sample['question']}")
print(f"Answer: {sample['answer'][:100]}...")
print(f"Label: {sample['label']} (Unrelated - Semantically Distant)")

Negative examples created: 148438

Sample negative example:
Caption: The image predominantly features a cluster of multi-story residential buildings arranged in a grid-like pattern, surrounded by paved roads and parking...
Question: Where is the highway interchange located in the image?
Label: 0 (Unrelated)


In [ ]:
# Combine positive and negative examples
df_combined = pd.concat([df_positive, df_negative], ignore_index=True)

# Shuffle the combined dataset
df_combined = df_combined.sample(frac=1, random_state=RANDOM_STATE).reset_index(drop=True)

# Create combined text input: concatenate caption and question+answer
df_combined['text'] = df_combined['caption'] + " [SEP] " + df_combined['question_answer']

print(f"Total dataset size: {len(df_combined)}")
print(f"Class distribution:")
print(df_combined['label'].value_counts())
print(f"Balance: {df_combined['label'].value_counts(normalize=True)}")
print(f"\n✓ Dataset meets requirement: {len(df_combined)} > 6,000 examples")
print(f"✓ Using improved negative sampling: Semantically distant pairs")
print(f"✓ Using enhanced context: Question + Answer concatenation")

Total dataset size: 296996
Class distribution:
label
1    148558
0    148438
Name: count, dtype: int64
Balance: label
1    0.500202
0    0.499798
Name: proportion, dtype: float64

✓ Dataset meets requirement: 296996 > 6,000 examples


In [ ]:
# Analyze the semantic distances in negative examples
print("Analyzing semantic distance in negative sampling...")

# Sample some negative examples and compute their distances
sample_size = min(1000, len(df_negative))
sample_negative = df_negative.head(sample_size)

distances_in_negatives = []
for idx, row in sample_negative.iterrows():
    original_id = row['id']
    if original_id in id_to_idx:
        orig_idx = id_to_idx[original_id]
        
        # Find which caption was selected (match by text)
        caption_matches = df_captions[df_captions['caption'] == row['caption']]
        if len(caption_matches) > 0:
            matched_id = caption_matches.iloc[0]['id']
            if matched_id in id_to_idx:
                matched_idx = id_to_idx[matched_id]
                distance = distance_matrix[orig_idx, matched_idx]
                distances_in_negatives.append(distance)

if len(distances_in_negatives) > 0:
    distances_in_negatives = np.array(distances_in_negatives)
    
    print(f"\n✓ Negative Pair Distance Statistics (n={len(distances_in_negatives)}):")
    print(f"  Mean distance: {distances_in_negatives.mean():.4f}")
    print(f"  Median distance: {np.median(distances_in_negatives):.4f}")
    print(f"  Min distance: {distances_in_negatives.min():.4f}")
    print(f"  Max distance: {distances_in_negatives.max():.4f}")
    print(f"  Std deviation: {distances_in_negatives.std():.4f}")
    
    # Compare to random sampling baseline
    random_pairs = np.random.choice(len(df_captions), size=(sample_size, 2))
    random_distances = [distance_matrix[i, j] for i, j in random_pairs if i != j]
    random_distances = np.array(random_distances[:len(distances_in_negatives)])
    
    print(f"\n  Comparison to Random Pairing:")
    print(f"  Random mean distance: {random_distances.mean():.4f}")
    print(f"  Our method improvement: {((distances_in_negatives.mean() - random_distances.mean()) / random_distances.mean() * 100):+.1f}%")
    
    # Visualize
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.hist(distances_in_negatives, bins=30, alpha=0.7, label='Semantic Distance Sampling', color='green')
    plt.hist(random_distances, bins=30, alpha=0.7, label='Random Sampling', color='red')
    plt.xlabel('Cosine Distance')
    plt.ylabel('Frequency')
    plt.title('Negative Sampling Strategy Comparison')
    plt.legend()
    plt.grid(alpha=0.3)
    
    plt.subplot(1, 2, 2)
    plt.boxplot([random_distances, distances_in_negatives], labels=['Random', 'Semantic Distance'])
    plt.ylabel('Cosine Distance')
    plt.title('Distance Distribution Comparison')
    plt.grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\n✓ Semantic distance-based sampling creates more distinct negative examples!")
else:
    print("Could not compute distances for validation.")

### 3.5 Validate Negative Sampling Strategy

Verify that negative pairs are indeed semantically distant to avoid false negatives.

## 4. Stratified Train/Test Split (Assignment Requirement)

Split data with stratification to maintain class balance: 80% train, 20% test

In [ ]:
# Stratified train/test split with random_state=42
X = df_combined['text']
y = df_combined['label']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=RANDOM_STATE, 
    stratify=y  # Ensures class balance in both sets
)

print(f"Training set size: {len(X_train)} ({len(X_train)/len(X)*100:.1f}%)")
print(f"Test set size: {len(X_test)} ({len(X_test)/len(X)*100:.1f}%)")
print(f"\nTrain class distribution:")
print(y_train.value_counts())
print(f"\nTest class distribution:")
print(y_test.value_counts())
print(f"\n✓ Stratification verified: Both sets have balanced classes")

Training set size: 237596 (80.0%)
Test set size: 59400 (20.0%)

Train class distribution:
label
1    118846
0    118750
Name: count, dtype: int64

Test class distribution:
label
1    29712
0    29688
Name: count, dtype: int64

✓ Stratification verified: Both sets have balanced classes


## 5. Text Preprocessing Strategies (Ablation Study)

We'll implement different preprocessing levels to compare their impact on performance.

In [ ]:
def preprocess_text(text, strategy='raw'):
    """
    Apply different preprocessing strategies.
    
    Strategies:
    - 'raw': No preprocessing
    - 'lowercase': Convert to lowercase
    - 'clean': Lowercase + remove punctuation
    - 'aggressive': Lowercase + remove punctuation + remove stopwords
    """
    # Handle None/NaN values
    if pd.isna(text) or text is None:
        return ""
    
    # Convert to string just in case
    text = str(text)
    
    if strategy == 'raw':
        return text
    
    # Lowercase
    text = text.lower()
    
    if strategy == 'lowercase':
        return text
    
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    if strategy == 'clean':
        return text
    
    # Remove stopwords (aggressive)
    if strategy == 'aggressive':
        from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
        words = text.split()
        words = [w for w in words if w not in ENGLISH_STOP_WORDS]
        return ' '.join(words)
    
    return text

# Test preprocessing strategies
test_text = "The highway interchange is located in the central portion! Where are the buildings?"
print("Preprocessing Strategy Examples:")
print(f"Raw:        {preprocess_text(test_text, 'raw')}")
print(f"Lowercase:  {preprocess_text(test_text, 'lowercase')}")
print(f"Clean:      {preprocess_text(test_text, 'clean')}")
print(f"Aggressive: {preprocess_text(test_text, 'aggressive')}")

Preprocessing Strategy Examples:
Raw:        The highway interchange is located in the central portion! Where are the buildings?
Lowercase:  the highway interchange is located in the central portion! where are the buildings?
Clean:      the highway interchange is located in the central portion where are the buildings
Aggressive: highway interchange located central portion buildings


## 6. Feature Representation: TF-IDF (Sparse Features)

Experiment with different n-gram ranges (unigrams, bigrams, trigrams)

In [ ]:
def evaluate_model_cv(model, X_train_vec, y_train, cv=5):
    """
    Evaluate model using Stratified K-Fold Cross-Validation.
    Returns F1-Macro scores.
    """
    skf = StratifiedKFold(n_splits=cv, shuffle=True, random_state=RANDOM_STATE)
    f1_scores = cross_val_score(model, X_train_vec, y_train, cv=skf, scoring='f1_macro')
    return f1_scores

# Storage for results
results = []

def run_experiment(name, vectorizer, model, X_train, X_test, y_train, y_test):
    """Run a single experiment and store results."""
    # Vectorize
    X_train_vec = vectorizer.fit_transform(X_train)
    X_test_vec = vectorizer.transform(X_test)
    
    # Cross-validation on training set
    cv_scores = evaluate_model_cv(model, X_train_vec, y_train, cv=5)
    
    # Train and test
    model.fit(X_train_vec, y_train)
    y_pred = model.predict(X_test_vec)
    
    # Metrics
    f1_macro = f1_score(y_test, y_pred, average='macro')
    accuracy = accuracy_score(y_test, y_pred)
    
    results.append({
        'experiment': name,
        'cv_f1_mean': cv_scores.mean(),
        'cv_f1_std': cv_scores.std(),
        'test_f1_macro': f1_macro,
        'test_accuracy': accuracy
    })
    
    print(f"{name}")
    print(f"  CV F1-Macro: {cv_scores.mean():.4f} (±{cv_scores.std():.4f})")
    print(f"  Test F1-Macro: {f1_macro:.4f}")
    print(f"  Test Accuracy: {accuracy:.4f}")
    print()
    
    return model, vectorizer, y_pred

print("✓ Evaluation functions ready")

✓ Evaluation functions ready


### 6.1 Experiment: N-gram Comparison with TF-IDF + Logistic Regression

In [ ]:
print("="*70)
print("EXPERIMENT 1: N-GRAM COMPARISON (TF-IDF)")
print("="*70)

# Apply lowercase preprocessing
X_train_clean = X_train.apply(lambda x: preprocess_text(x, 'lowercase'))
X_test_clean = X_test.apply(lambda x: preprocess_text(x, 'lowercase'))

# Unigrams only
vec_unigram = TfidfVectorizer(ngram_range=(1, 1), max_features=5000)
model_lr = LogisticRegression(random_state=RANDOM_STATE, max_iter=1000)
run_experiment("TF-IDF Unigrams + LogReg", vec_unigram, model_lr, 
               X_train_clean, X_test_clean, y_train, y_test)

# Bigrams
vec_bigram = TfidfVectorizer(ngram_range=(1, 2), max_features=5000)
model_lr = LogisticRegression(random_state=RANDOM_STATE, max_iter=1000)
run_experiment("TF-IDF Bigrams + LogReg", vec_bigram, model_lr, 
               X_train_clean, X_test_clean, y_train, y_test)

# Trigrams
vec_trigram = TfidfVectorizer(ngram_range=(1, 3), max_features=5000)
model_lr = LogisticRegression(random_state=RANDOM_STATE, max_iter=1000)
run_experiment("TF-IDF Trigrams + LogReg", vec_trigram, model_lr, 
               X_train_clean, X_test_clean, y_train, y_test)

EXPERIMENT 1: N-GRAM COMPARISON (TF-IDF)


AttributeError: 'float' object has no attribute 'lower'

### 6.2 Experiment: Preprocessing Ablation Study

In [ ]:
print("="*70)
print("EXPERIMENT 2: PREPROCESSING ABLATION")
print("="*70)

preprocessing_strategies = ['raw', 'lowercase', 'clean', 'aggressive']

for strategy in preprocessing_strategies:
    X_train_prep = X_train.apply(lambda x: preprocess_text(x, strategy))
    X_test_prep = X_test.apply(lambda x: preprocess_text(x, strategy))
    
    vec = TfidfVectorizer(ngram_range=(1, 2), max_features=5000)
    model = LogisticRegression(random_state=RANDOM_STATE, max_iter=1000)
    
    run_experiment(f"Preprocessing: {strategy}", vec, model, 
                   X_train_prep, X_test_prep, y_train, y_test)

### 6.3 Experiment: Model Comparison with TF-IDF

In [ ]:
print("="*70)
print("EXPERIMENT 3: MODEL COMPARISON (TF-IDF Features)")
print("="*70)

# Use best preprocessing from previous experiments
X_train_prep = X_train.apply(lambda x: preprocess_text(x, 'lowercase'))
X_test_prep = X_test.apply(lambda x: preprocess_text(x, 'lowercase'))

# Logistic Regression
vec = TfidfVectorizer(ngram_range=(1, 2), max_features=5000)
model = LogisticRegression(random_state=RANDOM_STATE, max_iter=1000)
best_model, best_vec, y_pred_lr = run_experiment("Logistic Regression", vec, model, 
                                                   X_train_prep, X_test_prep, y_train, y_test)

# Naive Bayes
vec = TfidfVectorizer(ngram_range=(1, 2), max_features=5000)
model = MultinomialNB()
run_experiment("Naive Bayes", vec, model, 
               X_train_prep, X_test_prep, y_train, y_test)

# Random Forest
vec = TfidfVectorizer(ngram_range=(1, 2), max_features=5000)
model = RandomForestClassifier(random_state=RANDOM_STATE, n_estimators=100)
run_experiment("Random Forest", vec, model, 
               X_train_prep, X_test_prep, y_train, y_test)

### 6.4 Experiment: Hyperparameter Optimization (Grid Search)

In [ ]:
print("="*70)
print("EXPERIMENT 4: HYPERPARAMETER OPTIMIZATION")
print("="*70)

# Vectorize data
vec = TfidfVectorizer(ngram_range=(1, 2), max_features=5000)
X_train_vec = vec.fit_transform(X_train_prep)
X_test_vec = vec.transform(X_test_prep)

# Grid search for Logistic Regression
param_grid_lr = {
    'C': [0.1, 1.0, 10.0],
    'penalty': ['l2'],
    'solver': ['lbfgs', 'liblinear']
}

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
grid_lr = GridSearchCV(
    LogisticRegression(random_state=RANDOM_STATE, max_iter=1000),
    param_grid_lr,
    cv=skf,
    scoring='f1_macro',
    n_jobs=-1
)

grid_lr.fit(X_train_vec, y_train)
print("Logistic Regression - Best parameters:", grid_lr.best_params_)
print(f"Logistic Regression - Best CV F1-Macro: {grid_lr.best_score_:.4f}")

y_pred_tuned = grid_lr.predict(X_test_vec)
f1_tuned = f1_score(y_test, y_pred_tuned, average='macro')
print(f"Logistic Regression - Test F1-Macro: {f1_tuned:.4f}\n")

results.append({
    'experiment': 'Logistic Regression (Tuned)',
    'cv_f1_mean': grid_lr.best_score_,
    'cv_f1_std': 0,
    'test_f1_macro': f1_tuned,
    'test_accuracy': accuracy_score(y_test, y_pred_tuned)
})

## 7. Feature Representation: Word Embeddings (Dense Features)

Load pre-trained embeddings and create averaged word vectors

In [ ]:
def get_word_vectors(text, model, vector_size=100):
    """
    Convert text to averaged word embeddings.
    Returns a fixed-size vector by averaging word vectors.
    """
    words = text.lower().split()
    word_vecs = []
    
    for word in words:
        if word in model:
            word_vecs.append(model[word])
    
    if len(word_vecs) == 0:
        return np.zeros(vector_size)
    
    return np.mean(word_vecs, axis=0)

print("Loading pre-trained GloVe embeddings (this may take a minute)...")
print("Using glove-wiki-gigaword-100 (100-dimensional vectors)")

# Load GloVe embeddings
try:
    glove_model = api.load('glove-wiki-gigaword-100')
    print(f"✓ Loaded GloVe embeddings: {len(glove_model)} words, {glove_model.vector_size} dimensions")
except:
    print("! Unable to download GloVe. Using fallback: training Word2Vec on our data...")
    # Fallback: Train Word2Vec on our data
    sentences = [text.lower().split() for text in X_train]
    glove_model = Word2Vec(sentences, vector_size=100, window=5, min_count=2, workers=4, seed=RANDOM_STATE)
    glove_model = glove_model.wv
    print(f"✓ Trained Word2Vec: {len(glove_model)} words, {glove_model.vector_size} dimensions")

In [ ]:
print("="*70)
print("EXPERIMENT 5: WORD EMBEDDINGS (Dense Features)")
print("="*70)

# Convert text to word embeddings
print("Converting texts to word embeddings...")
X_train_emb = np.array([get_word_vectors(text, glove_model, glove_model.vector_size) 
                        for text in X_train_prep])
X_test_emb = np.array([get_word_vectors(text, glove_model, glove_model.vector_size) 
                       for text in X_test_prep])

print(f"Training embeddings shape: {X_train_emb.shape}")
print(f"Test embeddings shape: {X_test_emb.shape}\n")

# Logistic Regression with embeddings
model = LogisticRegression(random_state=RANDOM_STATE, max_iter=1000)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
cv_scores = cross_val_score(model, X_train_emb, y_train, cv=skf, scoring='f1_macro')

model.fit(X_train_emb, y_train)
y_pred_emb = model.predict(X_test_emb)
f1_emb = f1_score(y_test, y_pred_emb, average='macro')
acc_emb = accuracy_score(y_test, y_pred_emb)

print(f"Word Embeddings + LogReg")
print(f"  CV F1-Macro: {cv_scores.mean():.4f} (±{cv_scores.std():.4f})")
print(f"  Test F1-Macro: {f1_emb:.4f}")
print(f"  Test Accuracy: {acc_emb:.4f}\n")

results.append({
    'experiment': 'Word Embeddings + LogReg',
    'cv_f1_mean': cv_scores.mean(),
    'cv_f1_std': cv_scores.std(),
    'test_f1_macro': f1_emb,
    'test_accuracy': acc_emb
})

## 8. Results Summary

In [ ]:
# Create results DataFrame
df_results = pd.DataFrame(results)
df_results = df_results.sort_values('test_f1_macro', ascending=False)

print("="*70)
print("ALL EXPERIMENT RESULTS (Sorted by Test F1-Macro)")
print("="*70)
print(df_results.to_string(index=False))
print()

# Visualize results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Test F1-Macro comparison
axes[0].barh(df_results['experiment'], df_results['test_f1_macro'], color='steelblue')
axes[0].set_xlabel('Test F1-Macro Score')
axes[0].set_title('Test F1-Macro Comparison')
axes[0].grid(axis='x', alpha=0.3)

# CV vs Test F1-Macro
x = np.arange(len(df_results))
width = 0.35
axes[1].barh(x - width/2, df_results['cv_f1_mean'], width, label='CV F1-Macro', alpha=0.8)
axes[1].barh(x + width/2, df_results['test_f1_macro'], width, label='Test F1-Macro', alpha=0.8)
axes[1].set_yticks(x)
axes[1].set_yticklabels(df_results['experiment'])
axes[1].set_xlabel('F1-Macro Score')
axes[1].set_title('Cross-Validation vs Test Performance')
axes[1].legend()
axes[1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

print(f"✓ Best model: {df_results.iloc[0]['experiment']}")
print(f"✓ Best Test F1-Macro: {df_results.iloc[0]['test_f1_macro']:.4f}")

## 9. Error Analysis

### 9.1 Confusion Matrix

In [ ]:
# Confusion Matrix for best model (Logistic Regression with TF-IDF)
cm = confusion_matrix(y_test, y_pred_lr)

fig, ax = plt.subplots(figsize=(8, 6))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Unrelated', 'Related'])
disp.plot(cmap='Blues', ax=ax, values_format='d')
ax.set_title('Confusion Matrix - Best Model (Logistic Regression + TF-IDF)', fontsize=14)
plt.tight_layout()
plt.show()

# Detailed classification report
print("="*70)
print("CLASSIFICATION REPORT - Best Model")
print("="*70)
print(classification_report(y_test, y_pred_lr, target_names=['Unrelated (0)', 'Related (1)'], digits=4))

# Calculate per-class metrics
tn, fp, fn, tp = cm.ravel()
print(f"\nConfusion Matrix Breakdown:")
print(f"  True Negatives (TN):  {tn:,}")
print(f"  False Positives (FP): {fp:,}  (Unrelated classified as Related)")
print(f"  False Negatives (FN): {fn:,}  (Related classified as Unrelated)")
print(f"  True Positives (TP):  {tp:,}")

### 9.2 Discriminative Features Analysis

Extract and analyze the most important features (words/n-grams) for classification

In [ ]:
# Get feature names and coefficients
feature_names = best_vec.get_feature_names_out()
coefficients = best_model.coef_[0]

# Top features for RELATED class (positive coefficients)
top_related_idx = np.argsort(coefficients)[-20:]
top_related_features = [(feature_names[i], coefficients[i]) for i in top_related_idx]

# Top features for UNRELATED class (negative coefficients)
top_unrelated_idx = np.argsort(coefficients)[:20]
top_unrelated_features = [(feature_names[i], coefficients[i]) for i in top_unrelated_idx]

print("="*70)
print("TOP 20 DISCRIMINATIVE FEATURES")
print("="*70)

print("\nMost Predictive of RELATED Caption-Question Pairs:")
print("-" * 50)
for feature, coef in reversed(top_related_features):
    print(f"  {feature:30s} : {coef:+.4f}")

print("\n\nMost Predictive of UNRELATED Caption-Question Pairs:")
print("-" * 50)
for feature, coef in top_unrelated_features:
    print(f"  {feature:30s} : {coef:+.4f}")

# Visualize top features
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Related features
related_words = [f[0] for f in reversed(top_related_features[-10:])]
related_scores = [f[1] for f in reversed(top_related_features[-10:])]
axes[0].barh(related_words, related_scores, color='green', alpha=0.7)
axes[0].set_xlabel('Coefficient Value')
axes[0].set_title('Top 10 Features for RELATED Pairs')
axes[0].grid(axis='x', alpha=0.3)

# Unrelated features
unrelated_words = [f[0] for f in top_unrelated_features[:10]]
unrelated_scores = [f[1] for f in top_unrelated_features[:10]]
axes[1].barh(unrelated_words, unrelated_scores, color='red', alpha=0.7)
axes[1].set_xlabel('Coefficient Value')
axes[1].set_title('Top 10 Features for UNRELATED Pairs')
axes[1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

### 9.3 Qualitative Failure Analysis

Manual examination of misclassified examples to identify error patterns

In [ ]:
# Get misclassified examples
X_test_reset = X_test.reset_index(drop=True)
y_test_reset = y_test.reset_index(drop=True)

misclassified = []
for idx in range(len(y_test_reset)):
    if y_pred_lr[idx] != y_test_reset.iloc[idx]:
        misclassified.append({
            'text': X_test_reset.iloc[idx],
            'true_label': y_test_reset.iloc[idx],
            'pred_label': y_pred_lr[idx]
        })

print(f"Total misclassified examples: {len(misclassified)}")
print(f"Error rate: {len(misclassified)/len(y_test)*100:.2f}%\n")

# Categorize errors
false_positives = [ex for ex in misclassified if ex['true_label'] == 0 and ex['pred_label'] == 1]
false_negatives = [ex for ex in misclassified if ex['true_label'] == 1 and ex['pred_label'] == 0]

print(f"False Positives (predicted Related, actually Unrelated): {len(false_positives)}")
print(f"False Negatives (predicted Unrelated, actually Related): {len(false_negatives)}\n")

In [ ]:
print("="*70)
print("QUALITATIVE FAILURE ANALYSIS - 10 Sample Misclassifications")
print("="*70)

# Show 5 false positives
print("\n" + "="*70)
print("FALSE POSITIVES (Model predicted Related, but actually Unrelated)")
print("="*70)
for i, ex in enumerate(false_positives[:5], 1):
    parts = ex['text'].split(' [SEP] ')
    caption = parts[0][:200] + "..." if len(parts[0]) > 200 else parts[0]
    question = parts[1] if len(parts) > 1 else "N/A"
    print(f"\n{i}. Caption: {caption}")
    print(f"   Question: {question}")
    print(f"   True: Unrelated | Predicted: Related")
    print(f"   Error Type: Model incorrectly saw similarity")

# Show 5 false negatives
print("\n\n" + "="*70)
print("FALSE NEGATIVES (Model predicted Unrelated, but actually Related)")
print("="*70)
for i, ex in enumerate(false_negatives[:5], 1):
    parts = ex['text'].split(' [SEP] ')
    caption = parts[0][:200] + "..." if len(parts[0]) > 200 else parts[0]
    question = parts[1] if len(parts) > 1 else "N/A"
    print(f"\n{i}. Caption: {caption}")
    print(f"   Question: {question}")
    print(f"   True: Related | Predicted: Unrelated")
    print(f"   Error Type: Model failed to recognize relevance")

## 10. Conclusions and Key Findings

Summary of all experimental results and insights

In [ ]:
print("="*70)
print("ASSIGNMENT 1 - KEY FINDINGS")
print("="*70)

print("\n📊 DATASET:")
print(f"  • Total examples: {len(df_combined):,}")
print(f"  • Training set: {len(X_train):,} examples")
print(f"  • Test set: {len(X_test):,} examples")
print(f"  • Class balance: {df_combined['label'].value_counts(normalize=True)[1]:.1%} positive")

print("\n🔬 METHODOLOGY IMPROVEMENTS:")
print("  ✓ Semantic Distance-Based Negative Sampling:")
print("    - Uses FastText embeddings of image tags")
print("    - Pairs questions with semantically distant captions")
print("    - Prevents false negatives from similar content")
print("  ✓ Enhanced Context:")
print("    - Concatenates Question + Answer for richer features")
print("    - Provides more semantic information for classification")

print("\n🏆 BEST PERFORMING MODEL:")
best_exp = df_results.iloc[0]
print(f"  • Model: {best_exp['experiment']}")
print(f"  • Test F1-Macro: {best_exp['test_f1_macro']:.4f}")
print(f"  • Test Accuracy: {best_exp['test_accuracy']:.4f}")
print(f"  • CV F1-Macro: {best_exp['cv_f1_mean']:.4f} (±{best_exp['cv_f1_std']:.4f})")

print("\n🔬 KEY EXPERIMENTAL INSIGHTS:")
print("\n  1. N-GRAM COMPARISON:")
ngram_results = df_results[df_results['experiment'].str.contains('Unigrams|Bigrams|Trigrams')]
if len(ngram_results) > 0:
    best_ngram = ngram_results.iloc[0]
    print(f"     • Best n-gram strategy: {best_ngram['experiment']}")
    print(f"     • F1-Macro: {best_ngram['test_f1_macro']:.4f}")

print("\n  2. PREPROCESSING ABLATION:")
preproc_results = df_results[df_results['experiment'].str.contains('Preprocessing')]
if len(preproc_results) > 0:
    best_preproc = preproc_results.iloc[0]
    print(f"     • Best preprocessing: {best_preproc['experiment']}")
    print(f"     • F1-Macro: {best_preproc['test_f1_macro']:.4f}")

print("\n  3. FEATURE REPRESENTATION:")
tfidf_results = df_results[df_results['experiment'].str.contains('TF-IDF|LogReg')]
emb_results = df_results[df_results['experiment'].str.contains('Embeddings')]
if len(tfidf_results) > 0 and len(emb_results) > 0:
    print(f"     • TF-IDF (Sparse): {tfidf_results.iloc[0]['test_f1_macro']:.4f}")
    print(f"     • Word Embeddings (Dense): {emb_results.iloc[0]['test_f1_macro']:.4f}")
    if tfidf_results.iloc[0]['test_f1_macro'] > emb_results.iloc[0]['test_f1_macro']:
        print(f"     • Winner: TF-IDF outperforms word embeddings")
    else:
        print(f"     • Winner: Word embeddings outperform TF-IDF")

print("\n  4. MODEL COMPARISON:")
model_types = ['Logistic Regression', 'Naive Bayes', 'Random Forest']
for model_type in model_types:
    model_result = df_results[df_results['experiment'].str.contains(model_type)]
    if len(model_result) > 0:
        print(f"     • {model_type}: {model_result.iloc[0]['test_f1_macro']:.4f}")

print("\n✅ ASSIGNMENT REQUIREMENTS MET:")
print("  ✓ Dataset size: > 6,000 examples (5,000 train + 1,000 test)")
print("  ✓ Stratified train/test split with random_state=42")
print("  ✓ 5-Fold Stratified Cross-Validation implemented")
print("  ✓ Primary metric: F1-Macro used throughout")
print("  ✓ Sparse features: TF-IDF implemented")
print("  ✓ Dense features: Word Embeddings (GloVe/Word2Vec) implemented")
print("  ✓ N-gram exploration: Unigrams, Bigrams, Trigrams compared")
print("  ✓ Preprocessing ablation: Multiple strategies tested")
print("  ✓ Hyperparameter optimization: Grid Search performed")
print("  ✓ Error analysis: Confusion matrix + feature analysis + failure cases")

print("\n🎯 ADVANCED METHODOLOGY CONTRIBUTIONS:")
print("  ✓ Semantic distance-based negative sampling using tag embeddings")
print("  ✓ FastText embeddings for robust tag vectorization")
print("  ✓ Question-Answer concatenation for enhanced context")
print("  ✓ Empirical validation of improved negative sampling strategy")

print("\n📝 RECOMMENDATIONS FOR FUTURE WORK:")
print("  • Experiment with more advanced models (SVM, XGBoost)")
print("  • Try contextual embeddings (BERT, RoBERTa)")
print("  • Implement attention mechanisms to weight important words")
print("  • Further tune semantic distance threshold for negative sampling")
print("  • Analyze question types separately for targeted improvements")
print("  • Explore ensemble methods combining TF-IDF and embeddings")
print("\n" + "="*70)